安装所需要的依赖

```
pip install transformers tqdm boto3 requests regex -q
```

In [6]:
from transformers import BertTokenizer, BertModel

model_version = 'bert-base-chinese'
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)
vocab = tokenizer.vocab
print("字典大小：", len(vocab))


字典大小： 21128


我们通过随机抽取查看一下词典中的数据对应

In [7]:
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(t, id))

token               index          
-------------------------
cnn                  9206
##lia               10336
##歪                 16696
##蟋                 19151
##椹                 16554
闹                    7317
専                    2197
##•                 13499
寞                    2174
##濛                 17145


1. 准备原始文本数据
2. 将原始文本转换为bert相关输入格式
3. 在bert之上加入新layer成下游任务模型
4. 训练该下游任务模型
5. 对新样本做推测

In [8]:
import os
import pandas as pd

# 对数据进行清洗
df_train = pd.read_csv("data/train.csv")
empty_title = (df_train['title2_zh'].isnull() \
              | df_train['title1_zh'].isnull() \
              | (df_train['title2_zh'] == '') \
              | (df_train['title2_zh'] == '0'))
df_train = df_train[~empty_title]

print("原始样本数量：", len(df_train))

原始样本数量： 320543


In [9]:
# 剔除过长的样本数据避免 bert 无法将整个输入序列放入记忆力不强的 GPU
MAX_LENGTH = 30
df_train = df_train[~(df_train.title1_zh.apply(lambda x: len(x)) > MAX_LENGTH)]
df_train = df_train[~(df_train.title2_zh.apply(lambda x: len(x)) > MAX_LENGTH)]

# 只采用1%训练数据
SAMPLE_FRAC = 0.01
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9527)

# 去除不必要的单位并重新民命标题内容
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

# idempotence, 将结果另存为 tsv 供 PyTorch 使用
df_train.to_csv("train.tsv", sep="\t", index=False)

print("训练样本数量：", len(df_train))
df_train.head()

训练样本数量： 2657


,text_a,text_b,label
0,苏有朋要结婚了，但网友觉得他还是和林心如比较合适,好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！,unrelated
1,爆料李小璐要成前妻了贾乃亮模仿王宝强一步到位、快刀斩乱麻！,李小璐要变前妻了？贾乃亮可能效仿王宝强当机立断，快刀斩乱麻！,agreed
2,为彩礼，母亲把女儿嫁给陌生男子，十年后再见面，母亲湿了眼眶,阿姨，不要彩礼是觉得你家穷，给你台阶下，不要以为我嫁不出去！,unrelated
3,猪油是个宝，一勺猪油等于十副药，先备起来再说,传承千百的猪油为何变得人人唯恐避之不及？揭开猪油的四大谣言！,unrelated
4,剖析：香椿，为什么会致癌？,香椿含亚硝酸盐多吃会致癌？测完发现是谣言,disagreed


由于样本数据中的unrelated 的数量为68% ,因此我们用的bert训练出的最少要超过68% 的baseline才可以：

In [10]:
df_train.label.value_counts() / len(df_train)

unrelated    0.679338
agreed       0.294317
disagreed    0.026346
Name: label, dtype: float64

对测试数据进行处理，满足格式要求

In [11]:
df_test = pd.read_csv("data/test.csv")
df_test = df_test.loc[:, ["title1_zh", "title2_zh", "id"]]
df_test.columns = ["text_a", "text_b", "Id"]
df_test.to_csv("test.tsv", sep="\t", index=False)

print("预测样本数据：", len(df_test))
df_test.head()

预测样本数据： 80126


,text_a,text_b,Id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191


In [12]:
ratio = len(df_test) / len(df_train)
print("测试数据样本树 / 训练数据样本书 = {:.1f} 倍".format(ratio))

测试数据样本树 / 训练数据样本书 = 30.2 倍


将数据内容转化为 dataset 相容的格式内容

In [13]:
"""
作为一个可以用来读取训练 / 测试集的 Dataset, 这是你需要彻底了解的部分，
此 Dataset 每次將 tsv 里的一组成对句子转化成 BERT 相容的格式，并回传3哥 tensors：
- tokens_tensor：两个句子合并后的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用来识别两个句子的界限 binary tensor
- label_tensor：将分类目标转化为类别索引 tensor, 如果是测试集则回传 None
"""
from torch.utils.data import Dataset
import torch
import pysnooper

class FakeNewsDataset(Dataset):
    # 读取之前初始化后的 tsv 并初始化一部分参数
    def __init__(self, mode, tokenizer):
        # 一般训练会需要 dev set
        assert mode in ["train", "test"]
        self.mode = mode
        # 大数据你会需要用 iterator = True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'agreed': 0, 'disagreed': 1, 'unrelated': 2}
        self.tokenizer = tokenizer

    # 定义回传一些训练 / 测试数据函数
    @pysnooper.snoop()
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, test_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)

        # 建立第一个句子的 bert tokens 并加入分割符号 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        print(tokens_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)

        # 第二个句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a

        # 将整个token 序列转化成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        # 将第一句包含 [SEP] 的 token 位置设为 0, 其他为 1 标示第二句
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, dtype=torch.long)

        return tokens_tensor, segments_tensor, label_tensor

    def __len__(self):
        return self.len

# 初始化一个专门读取训练样本的 Dataset 使用中文 bert 断词
trainset = FakeNewsDataset("train", tokenizer=tokenizer)

想要使用 bert ，就要使用bert能够看的懂的数据，我们需要彻底了解这个Dataset，接下来看一下格式的差异

In [14]:
# 选择第一个样本
sample_idx = 0

# 将原始文本拿出做对比
text_a, text_b, label = trainset.df.iloc[sample_idx].values

# 利用刚才建立好的 Dataset 取出转换后的 id tensors
token_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 将 token_tensor 还原成文本
tokens = tokenizer.convert_ids_to_tokens(token_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前后差异
print(f"""[原始文本]
句子1： {text_a}
句子2： {text_b}
分类：  {label}

-------------------

[Dataset 回传的 tensors]
token_tensor :  {token_tensor}

segment_tensor: {segments_tensor}

label_tensor: {label_tensor}

-------------------

[还原 token_tensors]
{combined_text}
""")


['苏', '有', '朋', '要', '结', '婚', '了', '，', '但', '网', '友', '觉', '得', '他', '还', '是', '和', '林', '心', '如', '比', '较', '合', '适']
[原始文本]
句子1： 苏有朋要结婚了，但网友觉得他还是和林心如比较合适
句子2： 好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！
分类：  unrelated

-------------------

[Dataset 回传的 tensors]
token_tensor :  tensor([ 101, 5722, 3300, 3301, 6206, 5310, 2042,  749, 8024,  852, 5381, 1351,
        6230, 2533,  800, 6820, 3221, 1469, 3360, 2552, 1963, 3683, 6772, 1394,
        6844,  102, 1962, 7318, 6057, 5310, 2042, 5314,  679, 2042, 3184, 4638,
        4912, 2269, 2803, 5709, 4413, 8024,  948, 7450, 4638, 4912, 2269, 2957,
        3717, 7027, 5010, 1526, 5722, 3300, 3301, 8013,  102])

segment_tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])

label_tensor: 2

-------------------

[还原 token_tensors]
[CLS]苏有朋要结婚了，但网友觉得他还是和林心如比较合适[SEP]好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！[SEP]



Source path:... <ipython-input-13-cd674cf6e081>
Starting var:.. self = <__main__.FakeNewsDataset object at 0x7faeb9f5d970>
Starting var:.. idx = 0
22:29:45.243533 call        25     # 定义回传一些训练 / 测试数据函数
22:29:45.243814 line        27     def __getitem__(self, idx):
22:29:45.243861 line        31         else:
New var:....... text_a = '苏有朋要结婚了，但网友觉得他还是和林心如比较合适'
New var:....... text_b = '好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！'
New var:....... label = 'unrelated'
22:29:45.244340 line        32             text_a, text_b, label = self.df.iloc[idx, :].values
New var:....... label_id = 2
22:29:45.244444 line        33             label_id = self.label_map[label]
New var:....... label_tensor = tensor(2)
22:29:45.244626 line        36         # 建立第一个句子的 bert tokens 并加入分割符号 [SEP]
New var:....... word_pieces = ['[CLS]']
22:29:45.245243 line        37         word_pieces = ["[CLS]"]
New var:....... tokens_a = ['苏', '有', '朋', '要', '结', '婚', '了', '，', '但', '网...'还', '是', '和', '林', '心', '如', '比', '较', '合', '

除了FakeNewDataset数据内容，以下内容为bert服务自己的NLP任务需要彻底搞明白的。


In [19]:
"""
当作可以一次回传一个mini-batch 的 DataLoader
这个 DataLoader 吃上面定一个 `FakeNewsDataset`.
回传训练 bert 时会需要 4 个 tensors:
- token_tensors: （batch_size, max_seq_len_in_batch）
- segment_tensors: (batch_size, max_seq_len_in_batch )
- masks_tensors: (batch_size, max_seq_len_in_batch )
- label_tensors: (batch_size)
"""


from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segment_tensors = [s[0] for s in samples]

    # 测试集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None

    # zero pad 到同一序列长度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segment_tensors = pad_sequence(segment_tensors, batch_first=True)

    # attention masks 将 token_tensors 设置不为 zero padding
    # 的位置设为1 让 bert 只需要关注这些位置的tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    return tokens_tensors, segment_tensors, masks_tensors, label_ids

# 初始化一个每次回传 64 个训练样本的 DataLoader
# 利用`collate_fn` 将 list of samples 合成一个 mini-batch
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [20]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape}
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")

Starting var:.. self = <__main__.FakeNewsDataset object at 0x7faeb9f5d970>
Starting var:.. idx = 0
22:33:34.692344 call        25     # 定义回传一些训练 / 测试数据函数
22:33:34.692601 line        27     def __getitem__(self, idx):
22:33:34.692651 line        31         else:
New var:....... text_a = '苏有朋要结婚了，但网友觉得他还是和林心如比较合适'
New var:....... text_b = '好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！'
New var:....... label = 'unrelated'
22:33:34.693154 line        32             text_a, text_b, label = self.df.iloc[idx, :].values
New var:....... label_id = 2
22:33:34.693240 line        33             label_id = self.label_map[label]
New var:....... label_tensor = tensor(2)
22:33:34.693369 line        36         # 建立第一个句子的 bert tokens 并加入分割符号 [SEP]
New var:....... word_pieces = ['[CLS]']
22:33:34.693712 line        37         word_pieces = ["[CLS]"]
New var:....... tokens_a = ['苏', '有', '朋', '要', '结', '婚', '了', '，', '但', '网...'还', '是', '和', '林', '心', '如', '比', '较', '合', '适']
22:33:34.694732 line        38         token

['苏', '有', '朋', '要', '结', '婚', '了', '，', '但', '网', '友', '觉', '得', '他', '还', '是', '和', '林', '心', '如', '比', '较', '合', '适']
['爆', '料', '李', '小', '璐', '要', '成', '前', '妻', '了', '贾', '乃', '亮', '模', '仿', '王', '宝', '强', '一', '步', '到', '位', '、', '快', '刀', '斩', '乱', '麻', '！']
['为', '彩', '礼', '，', '母', '亲', '把', '女', '儿', '嫁', '给', '陌', '生', '男', '子', '，', '十', '年', '后', '再', '见', '面', '，', '母', '亲', '湿', '了', '眼', '眶']
['猪', '油', '是', '个', '宝', '，', '一', '勺', '猪', '油', '等', '于', '十', '副', '药', '，', '先', '备', '起', '来', '再', '说']
['剖', '析', '：', '香', '椿', '，', '为', '什', '么', '会', '致', '癌', '？']
['你', '听', '说', '过', '[UNK]', '伪', '狂', '犬', '病', '[UNK]', '吗', '？', '真', '正', '的', '狂', '犬', '病', '人', '不', '会', '学', '狗', '叫']
['有', '钱', '人', '买', '房', '从', '不', '选', '这', '几', '层', '？', '开', '发', '商', '不', '会', '告', '诉', '你', '的', '秘', '密', '，', '看', '完', '才', '知', '多', '坑']
['真', '假', '？', '武', '安', '某', '商', '场', '家', '长', '因', '孩', '子', '争', '抢', '玩', '具', '大', '打', '出', '手', '致', '一', '人', '死', '亡',

除了label_ids 以外，其他3个tensors的每个样本的最后都为0， 这是因为每个样本的tokens序列基本上长度都会不懂，需要用paddings.
![](https://leemeng.tw/images/bert/from_raw_data_to_bert_compatible.jpg)